# Logo detection model 

In [1]:
import pandas as pd
import numpy as np

import os
import cv2
from PIL import Image
import urllib.request

import tqdm
import warnings
warnings.filterwarnings('ignore')
import random
import re

from matplotlib import pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import model_from_json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow.keras.utils

from sqlalchemy import create_engine, exc


Unzip and put all the data in one flat folder

In [ ]:
!unzip FlickrLogos32v2

In [ ]:
!python copy_images_to_flat_dir.py

In [2]:
PATH = "./FlickrLogos-v2/flat/jpg/"
len(os.listdir(PATH))

12951

In [3]:
df = pd.read_csv('./FlickrLogos-v2/all.txt', delimiter = ",", names= ['logo', 'jpg'])
df

,logo,jpg
0,google,462663740.jpg
1,google,2555224827.jpg
2,google,2969981253.jpg
3,google,5182026221.jpg
4,google,2297499608.jpg
...,...,...
8235,no-logo,4595735316.jpg
8236,no-logo,4649353862.jpg
8237,no-logo,3060617157.jpg
8238,no-logo,652497606.jpg


### Selecting beer logos

In [4]:
df = pd.read_csv('./FlickrLogos-v2/all.txt', delimiter = ",", names= ['logo', 'jpg'])
df_beer = df[(df.logo =='paulaner') |
             (df.logo =='guiness') | 
             (df.logo =='singha') |
             (df.logo =='fosters')| 
             (df.logo =='heineken')|
             (df.logo =='erdinger')|
             (df.logo =='carlsberg')|
             (df.logo =='becks')|
             (df.logo =='stellaartois')|
             (df.logo =='corona')|             
             (df.logo =='no-logo')]

In [5]:
df_beer.logo.value_counts()

no-logo         6000
erdinger          70
heineken          70
stellaartois      70
singha            70
paulaner          70
corona            70
guiness           70
carlsberg         70
becks             70
fosters           70
Name: logo, dtype: int64

We have class imbalance problem becaue of 6000 no-logo images. We will use image ImageDataGenerator to genarate more images from beer logos

In [ ]:
all_beers = df_beer.logo.unique()
datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest'        
       )
for logo in tqdm.tqdm(all_beers):
    df_logo = df[df.logo == logo]
    X = np.array([load_reshape_img(PATH + image) for image in df_logo.jpg])
    X = X.reshape(X.shape[0], 224, 224, 3)    
    i = 0
    for batch in datagen.flow(X, batch_size=70,
                              save_to_dir="augmented_images", 
                              save_prefix=logo, 
                              save_format='jpeg'):        
        i += 1
        if i > 5:
            break


In [ ]:
!cp /storage/logo_detection/augmented_images/* ./FlickrLogos-v2/flat/jpg/   # copying augmented images to the main data folder

In [37]:
def no_number(logo):
    label = re.findall('[a-z]+', logo)[0]
    return label
aug_img = os.listdir("augmented_images")
df_aug = pd.DataFrame()
df_aug['jpg'] = aug_img
df_aug['logo'] = df_aug['jpg'].apply(no_number)
df_beer = pd.concat([df_aug, df_beer])

Choosing only 7 brands to increase the accuracy

In [38]:
df_beer = df_beer[(df_beer.logo =='paulaner') |
                  (df_beer.logo =='guiness') | 
                  (df_beer.logo =='fosters')| 
                  (df_beer.logo =='heineken')|
                  (df_beer.logo =='carlsberg')|
                  (df_beer.logo =='becks')|
                  (df_beer.logo =='corona')|             
                  (df_beer.logo =='no-logo')]

In [39]:
df_beer.logo.value_counts()

no-logo      6000
paulaner      490
corona        490
fosters       490
guiness       490
carlsberg     490
heineken      490
becks         490
Name: logo, dtype: int64

### Inception-V3 Model

Load and reshape the images:

In [10]:
def load_reshape_img(image):
    img = load_img(image, target_size=(224, 224))
    x = img_to_array(img)/255.
    x = x.reshape((1,) + x.shape)
    return x


In [17]:
X = np.array([load_reshape_img(PATH + image) for image in tqdm.tqdm(df_beer.jpg)])
X = X.reshape(X.shape[0], 224, 224, 3)

100%|██████████| 9430/9430 [03:14<00:00, 48.37it/s] 


Hot encoding the labels:

In [15]:
keys = df_beer.logo.unique()
values = [i for i in range(0,len(keys))]
logo_dict = dict(zip(keys,values))
df_beer.logo.replace(logo_dict, inplace=True)
y = tensorflow.keras.utils.to_categorical(df_beer.logo, num_classes=8, dtype='float32')

Train and test split:

In [19]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=42)

Training the model:

In [25]:
model_v3 = InceptionV3(weights='imagenet', include_top=False, classes = 8, input_shape = (224, 224, 3))
new_layers = model_v3.output

new_layers = GlobalAveragePooling2D()(new_layers)

new_layers = Dense(1024, activation='relu')(new_layers)
new_layers = Dropout(0.5)(new_layers)
new_layers = BatchNormalization()(new_layers)

new_layers = Dense(8, activation='softmax')(new_layers)
model_v3 = Model(inputs=model_v3.inputs, outputs=new_layers)

# Freezing the first 51 layers
for layer in model_v3.layers[:52]:
    layer.trainable = False


In [26]:
model_v3.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy']) #Stochastic gradient descent optimizer.
results = model_v3.fit(Xtrain, ytrain, epochs=20, batch_size=8, validation_split=0.2)

Train on 5657 samples, validate on 1415 samples
Epoch 1/20
5657/5657 [==============================] - 65s 12ms/sample - loss: 1.2101 - accuracy: 0.6261 - val_loss: 0.3644 - val_accuracy: 0.8876
Epoch 2/20
5657/5657 [==============================] - 49s 9ms/sample - loss: 0.6410 - accuracy: 0.8153 - val_loss: 0.2198 - val_accuracy: 0.9378
Epoch 3/20
5657/5657 [==============================] - 49s 9ms/sample - loss: 0.4553 - accuracy: 0.8724 - val_loss: 0.1631 - val_accuracy: 0.9512
Epoch 4/20
5657/5657 [==============================] - 48s 9ms/sample - loss: 0.3186 - accuracy: 0.9072 - val_loss: 0.1299 - val_accuracy: 0.9618
Epoch 5/20
5657/5657 [==============================] - 49s 9ms/sample - loss: 0.2669 - accuracy: 0.9284 - val_loss: 0.1054 - val_accuracy: 0.9668
Epoch 6/20
5657/5657 [==============================] - 49s 9ms/sample - loss: 0.2404 - accuracy: 0.9406 - val_loss: 0.0895 - val_accuracy: 0.9739
Epoch 7/20
5657/5657 [==============================] - 49s 9ms/sampl

Saving the model and the weights:

In [10]:
def save_model(model):
    # serialize model to JSON
    with open(f"{model}.json", "w") as json_file:
        json_file.write(model_v3.to_json())

    # serialize weights to HDF5
    model_v3.save_weights(f"{model}.h5")
    print("Saved model to disk")
save_model('beer_logo_v3_aug_all')    

Saved model to disk


In [10]:
def load_logo_model():
    """
    load the saved trained logo detection model
    """
    # logging.critical("Loading logo detection model...")
    json_file = open('beer_logo_v3_aug_all.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("beer_logo_v3_aug_all.h5")
    # logging.critical("Model is ready.")
    return loaded_model
model_v3 = load_logo_model()

Classification report on the test data:

In [9]:
y_pred = model_v3.predict(Xtest, batch_size=64, verbose=0)
y_pred_bool = np.argmax(y_pred, axis=1)
ytest_bool = np.argmax(ytest, axis=1)
print(classification_report(ytest_bool, y_pred_bool, target_names=logo_dict.keys()))

              precision    recall  f1-score   support

     fosters       0.99      0.93      0.96       142
   carlsberg       0.97      0.97      0.97       117
       becks       1.00      1.00      1.00       122
     guiness       0.98      0.96      0.97       131
      corona       0.99      0.94      0.96       111
    heineken       0.99      0.97      0.98       110
    paulaner       0.99      0.97      0.98       120
     no-logo       0.99      1.00      0.99      1505

    accuracy                           0.99      2358
   macro avg       0.99      0.97      0.98      2358
weighted avg       0.99      0.99      0.99      2358



### Adding Hop House 13 logo to the model

In [24]:
datagen = ImageDataGenerator(    
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest'        
       )


In [40]:
def no_number(logo):
    label = re.findall('[a-z]+', logo)[:2]
    label = '-'.join(label)
    return label
def make_dataframe(path):
    new_logos = os.listdir(path)
    new_logos.remove('.ipynb_checkpoints')
    df_new_logo = pd.DataFrame()
    df_new_logo['jpg'] = new_logos
    df_new_logo['logo'] = df_new_logo['jpg'].apply(no_number)
    return df_new_logo
df_new_logo = make_dataframe("new_logos")


In [52]:
new_logos = ['hop-house']
for logo in tqdm.tqdm(new_logos):
    X = np.array([load_reshape_img('new_logos/' + image) for image in df_new_logo.jpg])
    X = X.reshape(X.shape[0], 224, 224, 3)    
    i = 0
    for batch in datagen.flow(X, batch_size=70,
                              save_to_dir="new_logos", 
                              save_prefix=logo, 
                              save_format='jpeg'):        
        i += 1
        if i > 5:
            break

100%|██████████| 2/2 [00:08<00:00,  4.49s/it]


In [53]:
!cp /storage/logo_detection/new_logos/* ./FlickrLogos-v2/flat/jpg/   # copying augmented images to the main data folder

In [43]:
df_new_logo = make_dataframe('new_logos')
df_beer = pd.concat([df_new_logo, df_beer])
df_beer.logo.value_counts()

no-logo      6000
paulaner      490
corona        490
fosters       490
guiness       490
carlsberg     490
heineken      490
becks         490
hop-house     243
Name: logo, dtype: int64

In [44]:
df_beer

,jpg,logo
0,hop-house (21).jpg,hop-house
1,hop-house_34_3916.jpeg,hop-house
2,hop-house_16_6480.jpeg,hop-house
3,hop-house_32_2514.jpeg,hop-house
4,hop-house_4_81.jpeg,hop-house
...,...,...
8235,4595735316.jpg,no-logo
8236,4649353862.jpg,no-logo
8237,3060617157.jpg,no-logo
8238,652497606.jpg,no-logo


In [45]:
df_beer.to_csv('df_beer.csv', index=False)

Re-train the model with the new logos:

In [14]:
X = np.array([load_reshape_img(PATH + image) for image in tqdm.tqdm(df_beer.jpg)])
X = X.reshape(X.shape[0], 224, 224, 3)

100%|██████████| 9673/9673 [02:32<00:00, 63.44it/s] 


In [15]:
keys = df_beer.logo.unique()
values = [i for i in range(0,len(keys))]
logo_dict = dict(zip(keys,values))
df_beer.logo.replace(logo_dict, inplace=True)
y = tensorflow.keras.utils.to_categorical(df_beer.logo, num_classes=9, dtype='float32')

In [16]:
model_v3 = InceptionV3(weights='imagenet', include_top=False, classes = 9, input_shape = (224, 224, 3))
new_layers = model_v3.output

new_layers = GlobalAveragePooling2D()(new_layers)

new_layers = Dense(1024, activation='relu')(new_layers)
new_layers = Dropout(0.5)(new_layers)
new_layers = BatchNormalization()(new_layers)

new_layers = Dense(9, activation='softmax')(new_layers)
model_v3 = Model(inputs=model_v3.inputs, outputs=new_layers)

In [17]:
# Freezing the first 51 layers
for layer in model_v3.layers[:52]:
    layer.trainable = False


In [19]:
model_v3.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy']) #Stochastic gradient descent optimizer.
results = model_v3.fit(X, y, epochs=20, batch_size=8, validation_split=0.2)

Train on 7738 samples, validate on 1935 samples
Epoch 1/20
7738/7738 [==============================] - 92s 12ms/sample - loss: 1.1389 - accuracy: 0.6352 - val_loss: 0.0239 - val_accuracy: 0.9979
Epoch 2/20
7738/7738 [==============================] - 74s 10ms/sample - loss: 0.4714 - accuracy: 0.8582 - val_loss: 0.0147 - val_accuracy: 0.9984
Epoch 3/20
7738/7738 [==============================] - 74s 10ms/sample - loss: 0.3320 - accuracy: 0.9066 - val_loss: 0.0187 - val_accuracy: 0.9964
Epoch 4/20
7738/7738 [==============================] - 74s 10ms/sample - loss: 0.2418 - accuracy: 0.9343 - val_loss: 0.0166 - val_accuracy: 0.9979
Epoch 5/20
7738/7738 [==============================] - 74s 10ms/sample - loss: 0.1800 - accuracy: 0.9484 - val_loss: 0.0131 - val_accuracy: 0.9979
Epoch 6/20
7738/7738 [==============================] - 75s 10ms/sample - loss: 0.1359 - accuracy: 0.9636 - val_loss: 0.0141 - val_accuracy: 0.9964
Epoch 7/20
7738/7738 [==============================] - 74s 10ms

In [28]:
save_model('beer_logo 1.0')  

Saved model to disk
